In [ ]:
import tweepy
import pandas as pd

In [ ]:
#Sign up on https://developer.twitter.com/ to create your app and get your credentials

consumer_key =" "
consumer_secret =" "
access_token =" " 
access_token_secret =" " 

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
number_of_tweets = 300
tweets = []
likes = []
time = []

for i in tweepy.Cursor(api.search, q=" ", geocode="latitude,longitude,radius", tweet_mode="extended").items(number_of_tweets):
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)

#In q=, write inside the " " your specific word or phrase that you want to search for
#In geocode=, replace "latitude, longitude, radius" with the area coordinates that you want to search for, or delete it if you do not need it.
#Radius units must be specified as km (for kilometers) or mi (for miles) 
#There is a helpful website to get coordinates https://www.mapdevelopers.com/draw-circle-tool.php 

In [ ]:
#Creating a dataframe 
df = pd.DataFrame({'tweets':tweets,'likes':likes,'time':time})

In [ ]:
#Removing RTs
df = df[~df.tweets.str.contains("RT")]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
#Normalization
df['tweets'] = df['tweets'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

#Changing tweets to lowercase
df['tweets'] = df['tweets'].str.lower()

#Removing words with less than 3 characters 
df['tweets'] = df['tweets'].str.replace(r'\b(\w{1,3})\b', '',regex=True)


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import spacy
nlp= spacy.load('es_core_news_sm')
import seaborn as sns

#If you need the english pipeline replace "es_core_news_sm" with "en_core_web_sm"

In [ ]:
#Splitting all the sentences
list_of_sentences = [sentence for sentence in df.tweets]

lines = []

for sentence in list_of_sentences:
    words = sentence.split()
    for w in words:
        lines.append(w)


In [ ]:
#Removing punctuation
lines = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in lines]

lines2 = []

for word in lines:
    if word !='':
        lines2.append(word)


In [ ]:
#Removing stop words
stem2 = []

for word in lines2:
    if word not in nlp.Defaults.stop_words:
        stem2.append(word)


In [ ]:
df2 = pd.DataFrame(stem2)

df2 = df2[0].value_counts()

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='black', 
                max_words=1000,
                relative_scaling=0.21,
                min_font_size = 10).generate_from_frequencies(df2)


In [ ]:
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
#Exporting the graph
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud.png', dpi=1000)